<a href="https://colab.research.google.com/github/arzss-code/Object-Tracking-YOLOv8/blob/main/object_tracking_with_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Numpy versi yang sesuai

In [ ]:
!pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 79.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.8 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
xarray 2025.3.1 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
blosc2 3.5.0 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
bigframes 2.8.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompat

### Menghubungkan Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Instalasi library Ultralytics

In [ ]:
!pip install ultralytics

# Memastikan OpenCV terinstal dengan benar
!pip install opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

### Setup Proyek (Download & Ekstrak Library)

In [ ]:
# Unduh file ZIP dari Google Drive
print("\nMengunduh deep_sort_pytorch.zip...")
!gdown "https://drive.google.com/uc?id=11ZSZcG-bcbueXZC3rN08CM0qqX3eiHxf&confirm=t"

# Unzip file tersebut. Kita gunakan -q agar output tidak terlalu panjang
print("\nMengekstrak file...")
!unzip -q deep_sort_pytorch.zip

# Tambahkan path ke Python agar bisa di-import
import sys
sys.path.append('/content/deep_sort_pytorch')

print("\n\nSETUP SELESAI. Semua file dari ZIP sudah siap.")


Mengunduh deep_sort_pytorch.zip...
Downloading...
From: https://drive.google.com/uc?id=11ZSZcG-bcbueXZC3rN08CM0qqX3eiHxf&confirm=t
To: /content/deep_sort_pytorch.zip
100% 43.1M/43.1M [00:00<00:00, 173MB/s]

Mengekstrak file...


SETUP SELESAI. Semua file dari ZIP sudah siap.


### Implementasi Kode (Deteksi, Tracking, Counting)

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from IPython.display import display, HTML, clear_output
from base64 import b64encode
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
from collections import deque, defaultdict
import math
import shutil

# --- KONFIGURASI UTAMA ---
CONFIG = {
    "YOLO_MODEL": "yolov8l.pt",
    "DEEPSORT_MAX_AGE": 30,
    "COUNTING_LINE_Y": 380, # <-- HANYA SATU GARIS SEKARANG
    "PPM_RATIO": 10.0,
    "VIDEO_FPS": 30.0,
    "ICON_SIZE": 35
}

# --- INISIALISASI VARIABEL GLOBAL ---
data_deque = {}; icons = {}; counts_up = defaultdict(int); counts_down = defaultdict(int)
model = None; cap = None; fps = 30.0

# --- WARNA KELAS KENDARAAN (BGR) ---
CLASS_COLORS = {
    'car': (0, 0, 255),
    'motorcycle': (0, 255, 0),
    'bus': (255, 165, 0),
    'truck': (255, 0, 255)
}

# --- FUNGSI-FUNGSI BANTUAN ---
def load_icons():
    global icons
    icon_path = '/content/drive/MyDrive/Proyek_Object_Tracking/icons/'
    icon_names = {'car': 'car.png', 'motorcycle': 'motorcycle.png', 'bus': 'bus.png', 'truck': 'truck.png'}
    for name, filename in icon_names.items():
        path = os.path.join(icon_path, filename)
        if os.path.exists(path):
            icon = cv2.imread(path, -1)
            if icon is not None: icons[name] = cv2.resize(icon, (CONFIG["ICON_SIZE"], CONFIG["ICON_SIZE"]))
        else: print(f"Peringatan: Ikon tidak ditemukan di {path}")

def overlay_icon(background, icon, x, y):
    h, w = icon.shape[:2]
    if y < 0 or x < 0 or y + h > background.shape[0] or x + w > background.shape[1]: return
    if icon.shape[2] == 4:
        alpha = icon[:, :, 3] / 255.0; alpha_inv = 1.0 - alpha
        for c in range(3): background[y:y+h, x:x+w, c] = (alpha * icon[:, :, c] + alpha_inv * background[y:y+h, x:x+w, c])

def draw_osd(img, counts_up, counts_down):
    osd_x, osd_y, pad = 20, 20, 10
    line_height = CONFIG["ICON_SIZE"] + pad

    total_items = 2 + len(counts_up) + len(counts_down)
    osd_h = max(150, (total_items * line_height) + pad) # Ukuran dinamis
    osd_w = 380

    # Pastikan box OSD tidak keluar dari frame
    if osd_y + osd_h > img.shape[0]: osd_h = img.shape[0] - osd_y
    if osd_x + osd_w > img.shape[1]: osd_w = img.shape[1] - osd_x

    sub_img = img[osd_y:osd_y+osd_h, osd_x:osd_x+osd_w]
    black_rect = np.ones(sub_img.shape, dtype=np.uint8) * 0
    res = cv2.addWeighted(sub_img, 0.6, black_rect, 0.4, 1.0)
    img[osd_y:osd_y+osd_h, osd_x:osd_x+osd_w] = res

    curr_y = osd_y + pad
    cv2.putText(img, "ARAH NAIK", (osd_x + pad, curr_y + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
    curr_y += line_height

    for cls, val in counts_up.items():
        obj_name = model.names[cls]
        if obj_name in icons:
            overlay_icon(img, icons[obj_name], osd_x + pad, curr_y)
        text = f": {obj_name.capitalize()} - {val}"
        cv2.putText(img, text, (osd_x + pad + CONFIG["ICON_SIZE"], curr_y + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        curr_y += line_height

    cv2.putText(img, "ARAH TURUN", (osd_x + pad, curr_y + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    curr_y += line_height

    for cls, val in counts_down.items():
        obj_name = model.names[cls]
        if obj_name in icons:
            overlay_icon(img, icons[obj_name], osd_x + pad, curr_y)
        text = f": {obj_name.capitalize()} - {val}"
        cv2.putText(img, text, (osd_x + pad + CONFIG["ICON_SIZE"], curr_y + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        curr_y += line_height

def draw_boxes_and_estimate_speed(img, bbox, names, object_ids, identities=None):
    for i, box in enumerate(bbox):
        x1, y1, x2, y2 = [int(i) for i in box]
        id = int(identities[i]) if identities is not None else 0

        obj_id = int(object_ids[i])
        obj_name = names[obj_id]
        color = CLASS_COLORS.get(obj_name, (200, 200, 200))

        if id not in data_deque: data_deque[id] = deque(maxlen=64)
        center = (int((x1 + x2) / 2), int((y1 + y2) / 2))
        data_deque[id].appendleft({'center': center, 'time': cap.get(cv2.CAP_PROP_POS_FRAMES)})

        speed_kmh = 0
        if len(data_deque[id]) >= 10:
            p1, t1 = data_deque[id][0]['center'], data_deque[id][0]['time']
            p2, t2 = data_deque[id][9]['center'], data_deque[id][9]['time']
            if t1 != t2:
                pixel_dist = math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)
                meter_dist = pixel_dist / CONFIG["PPM_RATIO"]
                time_delta = (t1 - t2) / fps
                if time_delta > 0: speed_kmh = (meter_dist / time_delta) * 3.6

        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

        label_id = f"{obj_name.capitalize()} ID:{id}"
        label_speed = f"{speed_kmh:.1f} km/j"

        # Mengurangi ukuran font dan ketebalan garis
        font_scale = 0.5
        thickness = 1

        (w_id, h_id), _ = cv2.getTextSize(label_id, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)
        (w_sp, h_sp), _ = cv2.getTextSize(label_speed, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)
        label_bg_w = max(w_id, w_sp)

        # Mengurangi jarak antar teks dan kotak
        text_padding = 5
        cv2.rectangle(img, (x1, y1 - text_padding - h_id - h_sp), (x1 + label_bg_w + text_padding, y1), color, -1)

        # Mengubah posisi teks sesuai dengan ukuran font baru dan padding
        cv2.putText(img, label_id, (x1 + text_padding // 2, y1 - text_padding - h_sp), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255,255,255), thickness)
        cv2.putText(img, label_speed, (x1 + text_padding // 2, y1 - text_padding // 2), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255,255,255), thickness)

        for j in range(1, len(data_deque[id])):
            if data_deque[id][j-1] and data_deque[id][j]:
                thickness = int(np.sqrt(64 / float(j + 1)) * 1.5)
                cv2.line(img, data_deque[id][j-1]['center'], data_deque[id][j]['center'], color, thickness)

from utils.parser import get_config
from deep_sort import DeepSort

def show_video(video_path, width=800):
    try:
        video_file = open(video_path, "rb").read()
        video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
        return HTML(f'<video width="{width}" controls><source src="{video_url}"></video>')
    except FileNotFoundError: return HTML(f"<p>Error: File tidak ditemukan di '{video_path}'.</p>")

DRIVE_PATH = '/content/drive/MyDrive/Proyek_Object_Tracking/'
VIDEO_DIR = os.path.join(DRIVE_PATH, 'videos/')
OUTPUT_DIR = os.path.join(DRIVE_PATH, 'output/')
os.makedirs(OUTPUT_DIR, exist_ok=True); os.makedirs(VIDEO_DIR, exist_ok=True)
try:
    video_files = [f for f in os.listdir(VIDEO_DIR) if f.endswith(('.mp4', '.avi'))]
    if not video_files: video_files = ['Tidak ada video.']
except FileNotFoundError: video_files = ['Error: Folder tidak ditemukan']
video_selector = widgets.Dropdown(options=video_files, description='Pilih Video:')
conf_slider = widgets.FloatSlider(value=0.5, min=0.1, max=1.0, step=0.05, description='Confidence:')
run_button = widgets.Button(description='Mulai Proses', button_style='success', icon='play')
output_widget = widgets.Output()

def run_processing(b):
    global model, cap, fps, data_deque, counts_up, counts_down
    with output_widget:
        clear_output(wait=True); print("Mempersiapkan pemrosesan...")
        load_icons(); data_deque.clear(); counts_up.clear(); counts_down.clear()

        video_name = video_selector.value
        if video_name.startswith('Tidak ada') or video_name.startswith('Error'): return print("Harap pilih file video yang valid.")

        DRIVE_VIDEO_OUT_PATH = os.path.join(OUTPUT_DIR, f"hasil_{video_name}")
        LOCAL_VIDEO_OUT_PATH = f"/content/hasil_{video_name}"
        VIDEO_IN_PATH = os.path.join(VIDEO_DIR, video_name)
        model = YOLO(CONFIG["YOLO_MODEL"])
        cfg = get_config(); cfg.merge_from_file("deep_sort_pytorch/configs/deep_sort.yaml")
        tracker = DeepSort(cfg.DEEPSORT.REID_CKPT, max_age=CONFIG["DEEPSORT_MAX_AGE"], use_cuda=True)

        cap = cv2.VideoCapture(VIDEO_IN_PATH)
        w, h = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS) if cap.get(cv2.CAP_PROP_FPS) > 0 else CONFIG["VIDEO_FPS"]
        out = cv2.VideoWriter(LOCAL_VIDEO_OUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

        counted_ids = set()
        print(f"Memproses video: {video_name}...")

        while cap.isOpened():
            success, frame = cap.read()
            if not success: break

            # Gambar garis hitung standar (kuning) di setiap frame
            cv2.line(frame, (0, CONFIG["COUNTING_LINE_Y"]), (w, CONFIG["COUNTING_LINE_Y"]), (0, 255, 255), 2)

            results = model(frame, conf=conf_slider.value, classes=[2, 3, 5, 7], verbose=False)
            xywhs, confs, clss = [], [], []
            for result in results:
                for box in result.boxes:
                    x1,y1,x2,y2 = box.xyxy[0].cpu().numpy()
                    xywhs.append([(x1+x2)/2, (y1+y2)/2, x2-x1, y2-y1])
                    confs.append(float(box.conf[0]))
                    clss.append(int(box.cls[0]))

            if len(xywhs) > 0:
                tracks = tracker.update(np.array(xywhs), np.array(confs), np.array(clss), frame)
                if len(tracks) > 0:
                    draw_boxes_and_estimate_speed(frame, tracks[:,:4], model.names, tracks[:,5].astype(int), tracks[:,4])

                    for track in tracks:
                        track_id, obj_id = int(track[4]), int(track[5])

                        if track_id not in counted_ids and len(data_deque[track_id]) > 1:
                            current_y = data_deque[track_id][0]['center'][1]
                            prev_y = data_deque[track_id][1]['center'][1]
                            line_y = CONFIG["COUNTING_LINE_Y"]

                            # Logika Hitung & Efek Visual Garis
                            if prev_y < line_y and current_y >= line_y: # Arah Turun
                                counts_down[obj_id] += 1
                                counted_ids.add(track_id)
                                cv2.line(frame, (0, line_y), (w, line_y), (0, 0, 255), 4) # Garis flash merah
                            elif prev_y > line_y and current_y <= line_y: # Arah Naik
                                counts_up[obj_id] += 1
                                counted_ids.add(track_id)
                                cv2.line(frame, (0, line_y), (w, line_y), (0, 0, 255), 4) # Garis flash merah

            draw_osd(frame, counts_up, counts_down)
            out.write(frame)

        cap.release()
        out.release()
        print("Pemrosesan video selesai.")

        print("Menyalin video hasil ke Google Drive Anda...")
        shutil.copy(LOCAL_VIDEO_OUT_PATH, DRIVE_VIDEO_OUT_PATH)
        print(f"✅ Video berhasil disimpan di: {DRIVE_VIDEO_OUT_PATH}")

        print("\n--- Analisis Data ---")
        # (Sisa kode analisis dan display video tidak berubah)
        if sum(counts_up.values()) > 0 or sum(counts_down.values()) > 0:
            print("Statistik Arah Naik:", dict(counts_up)); print("Statistik Arah Turun:", dict(counts_down))
        else: print("Tidak ada kendaraan yang terhitung.")

        print("\n--- Video Hasil ---")
        display(show_video(DRIVE_VIDEO_OUT_PATH))

run_button.on_click(run_processing)
display(widgets.VBox([video_selector, conf_slider, run_button, output_widget]))

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
